In [1]:
# Utilities
import matplotlib.pyplot as plt
import pandas as pd
import getpass
import requests

from sentinelhub import (
    SHConfig,
    DataCollection,
    SentinelHubCatalog,
    SentinelHubRequest,
    SentinelHubStatistical,
    BBox,
    bbox_to_dimensions,
    CRS,
    MimeType,
    Geometry,
)

from utils import plot_image

In [2]:
def get_open_elevation(lat, lon):
    query = f'https://api.open-elevation.com/api/v1/lookup?locations={lat},{lon}'
    response = requests.get(query).json()
    # Check if the response contains results
    if 'results' in response:
        return response['results'][0]['elevation']
    else:
        return None

In [5]:
import folium
from shapely.geometry import Polygon
from sentinelhub import Geometry, CRS
import random
import branca.colormap as cm

# Function to generate square polygons of 10x10 km in the given area
def generate_squares(lower_left, upper_right, step=0.1):
    lat_start, lon_start = lower_left
    lat_end, lon_end = upper_right
    
    lat = lat_start
    while lat + step < lat_end:
        lon = lon_start
        while lon + step < lon_end:
            square = Polygon([
                (lon, lat),
                (lon + step, lat),
                (lon + step, lat + step),
                (lon, lat + step),
                (lon, lat)
            ])
            yield square
            lon += step
        lat += step

# Define the corners of the area of interest
lower_left = (44.424300, 8.911724)
upper_right = (45.773590, 11.814051)

# 10 km in degrees latitude (approximate)
step = 10 / 111

# Generate the geometries for each square
geometries = []
for square in generate_squares(lower_left, upper_right, step):
    polygon_geometry = Geometry(geometry={
        "type": "Polygon",
        "coordinates": [list(square.exterior.coords)]
    }, crs=CRS.WGS84)
    geometries.append(polygon_geometry)
random_values = [random.random() for _ in geometries]
colormap = cm.LinearColormap(colors=['green', 'red'], vmin=0, vmax=1)
# Create a folium map
m = folium.Map(location=[(lower_left[0] + upper_right[0]) / 2, (lower_left[1] + upper_right[1]) / 2], zoom_start=8)

# Add polygons to the map
for geom, value in zip(geometries, random_values):
    style_function = lambda x, col=colormap(value): {'fillColor': col, 'color': 'black', 'weight': 1, 'fillOpacity': 0.5}
    # Create a GeoJson object and add it to the map
    gj = folium.GeoJson(data=geom.geojson, style_function=style_function)
    gj.add_to(m)

# Save to an HTML file
m.save('map.html')
m

In [7]:
evalscript_ndvi = """
//VERSION=3
function setup() {
  return {
    input: [{
      bands: [
        "B04",
        "B08",
        "dataMask"
      ]
    }],
    output: [
      {
        id: "ndvi",
        bands: 1
      },
      {
        id: "dataMask",
        bands: 1
      }]
  };
}

function evaluatePixel(samples) {
    let index = (samples.B08 - samples.B04) / (samples.B08+samples.B04);
    return {
        ndvi: [index],
        dataMask: [samples.dataMask],
    };
}

"""
evalscript_ndwi = """
//VERSION=3
function setup() {
  return {
    input: [{
      bands: [
        "B03", // Green band
        "B08", // NIR band
        "dataMask" // Mask to exclude the non-valid pixels
      ]
    }],
    output: [
      {
        id: "ndwi",
        bands: 1,
        sampleType: "FLOAT32"
      },
      {
        id: "dataMask",
        bands: 1
      }
    ]
  };
}

function evaluatePixel(samples) {
    let ndwi = (samples.B03 - samples.B08) / (samples.B03 + samples.B08);
    return {
        ndwi: [ndwi],
        dataMask: [samples.dataMask],
    };
}
"""




In [8]:
width_in_pixels = round(7360.86 / 200)  # Resulting in a whole number
height_in_pixels = round(16686.9 / 200)  # Resulting in a whole number
combined_df = pd.DataFrame(columns=['geometry_index', 'year', 'ndvi_mean', 'ndwi_mean', 'avg_elevation'])
for i, geometry in enumerate(geometries):
    request_ndvi = SentinelHubStatistical(
        aggregation=SentinelHubStatistical.aggregation(
            evalscript=evalscript_ndvi,
            time_interval=("2020-01-01T00:00:00Z", "2020-12-30T23:59:59Z"),
            aggregation_interval="P1D",
            size=[width_in_pixels, height_in_pixels],

        ),
        input_data=[
            SentinelHubStatistical.input_data(
                DataCollection.SENTINEL2_L1C.define_from(
                    name="s2l1c", service_url="https://sh.dataspace.copernicus.eu"
                ),
                other_args={"dataFilter": {"maxCloudCoverage": 10}},
            ),
        ],
        geometry=geometry,
        config=config,
    )
    request_ndwi = SentinelHubStatistical(
    aggregation=SentinelHubStatistical.aggregation(
        evalscript=evalscript_ndwi,
        time_interval=("2020-01-01T00:00:00Z", "2020-12-30T23:59:59Z"),
        aggregation_interval="P1D",
        size=[width_in_pixels, height_in_pixels],
    ),
    input_data=[
        SentinelHubStatistical.input_data(
            DataCollection.SENTINEL2_L1C.define_from(
                name="s2l1c", service_url="https://sh.dataspace.copernicus.eu"
            ),
            other_args={"dataFilter": {"maxCloudCoverage": 10}},
        ),
    ],
    geometry=geometry,
    config=config,
    )
    response_ndvi = request_ndvi.get_data()
    response_ndwi = request_ndwi.get_data()
    # result_ndvi = read_acquisitions_stats(response_ndvi[0]["data"])
    # result_ndwi = read_acquisitions_stats(response_ndwi[0]["data"])
    # Assume response_ndvi and response_ndwi are now populated with data
    # Convert results to DataFrame
    df_ndvi = pd.DataFrame(read_acquisitions_stats(response_ndvi[0]['data']))
    df_ndwi = pd.DataFrame(read_acquisitions_stats(response_ndwi[0]['data']))

    # Convert the 'date' column to datetime format to ease processing
    df_ndvi['date'] = pd.to_datetime(df_ndvi['date'])
    df_ndwi['date'] = pd.to_datetime(df_ndwi['date'])

    # Group by year and calculate the mean of the means for each year
    yearly_ndvi = df_ndvi.groupby(df_ndvi['date'].dt.year)['ndvi_mean'].mean().reset_index()
    yearly_ndwi = df_ndwi.groupby(df_ndwi['date'].dt.year)['ndwi_mean'].mean().reset_index()

    # Combine the yearly means into a single dataframe
    yearly_combined = pd.merge(yearly_ndvi, yearly_ndwi, on='date', how='outer', suffixes=('_ndvi', '_ndwi'))
    yearly_combined['geometry_index'] = i
    yearly_combined['avg_elevation'] = average_elevations[i]  # Assuming average_elevations list is already populated

    # Rename the columns to reflect the data correctly
    yearly_combined.rename(columns={'date': 'year'}, inplace=True)

    # Append the yearly combined data to the main combined dataframe
    combined_df = pd.concat([combined_df, yearly_combined], ignore_index=True)

# Convert the 'year' column back to a date format if needed
combined_df['year'] = pd.to_datetime(combined_df['year'], format='%Y').dt.to_period('Y')

# Now 'combined_df' contains the combined data for all geometries with the average NDVI, NDWI, and elevation per year
print(combined_df)

NameError: name 'config' is not defined

In [ ]:
combined_df.to_csv("test.csv")